In [1]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-m.json'); 
var kreativesFeld = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 26  
Gruppenanzahl: 5

In [2]:
var groups = 5;

In [5]:
// 100x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < 100; run++) {
    
    // Calculate groups with monte carlo
    runs.push(kreativesFeld.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

// Run async tasks in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        var highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

[
    35.4741877565865, 16.700402491362027, 49.989476981764994,  35.21243541232937,
  50.066924721417834, 36.019664540978354,  53.01492615213546, 28.566635468039095,
   26.97938700589222, 1.4537387084258313, 27.615126456268555,  23.36006539187278,
   40.54570398518234,  39.71415232114636, 33.820209857981254,  32.84755687030099,
  10.084540814943722, 21.406968766768728, 15.146104188727378, 20.454143628221004,
   55.96564447126114,  25.59539192222752,  45.32327349922939, 31.216138083187005,
   59.27548085518988, 17.697284846714822,  25.94322316534904,  37.33434843449868,
   35.02339583124758, 10.463788913028665, 34.233304765695095,  32.03377096877078,
  41.745231973013915, 26.203977945135378, 27.739094996332668,  19.90350123238713,
  19.235988939420167,  40.73402236190613, 40.226363818019095,  42.18100830137344,
   23.75691027806333,  38.73261445524189,  42.67088784075841,  34.37669852016698,
   59.90036448275124,  37.12726303724277, 41.020745583162466,  14.63184556401535,
   42.80410925

In [6]:
// 100x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

// Run algorithm x times
for(let run = 0; run < 100; run++) {
    runs.push(kreativesFeld.genetic(input, groups).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

[
   12.719520243471749,  36.08651921029107, 29.931463874836325,
    44.53570947218742, 24.360158401970423, 12.303508916191447,
   12.470665008084055, 60.650426400547374, 36.358625713443914,
    39.53163344635523,  43.13770889084569, 1.3369385952599657,
    35.10955688739223, 19.722907370769818,   5.41237913130837,
    6.713029438234997,  37.40425982037047, 15.454294272558428,
    32.22639371733955,  38.94231922651336,  45.77950460067113,
   25.527918133505565, 37.357655897282555,  6.749132343631061,
   22.293927682179586,  31.70144371307859,  49.72837136801763,
    50.61005989542048,  39.98524669310207,  52.76205708917065,
   28.399510456278662,  36.69469706282494, 37.505360363540674,
   13.941720409820036,   33.8863966482393,  8.068285358697523,
   16.011720959976728, 24.197484584677937,  24.03774849438092,
    23.80529812654015, 43.279882205004924, 36.510122055111395,
    44.13856277824863, 33.770235121395885,  29.97696752340204,
   26.139096247176894,  53.91864805661302, 43.7781025

## Vergleich Algorithmen

In [7]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);

In [8]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [9]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();

## Darstellung 2. Bestes Ergebnis

In [10]:
console.log(winner.combination);

[
  4, 2, 5, 3, 4, 1, 3, 1,
  4, 2, 1, 4, 5, 5, 5, 1,
  1, 5, 3, 2, 2, 2, 1, 3,
  3, 4
]
